In [1]:
import torch

In [2]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")


In [3]:
device

device(type='cuda', index=1)

In [4]:
import xarray as xr
import gcsfs
import numpy as np

fs = gcsfs.GCSFileSystem(token="anon")

store = fs.get_mapper('gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr')
ds = xr.open_zarr(store=store, consolidated=True)

# df_era5 = ds.specific_humidity.sel( {'time':np.arange('2018-01-01','2023-01-11',np.timedelta64(1,'D'),dtype='datetime64[ns]')})


In [5]:
ds

<xarray.Dataset> Size: 89TB
Dimensions:                                           (time: 93544,
                                                       latitude: 721,
                                                       longitude: 1440,
                                                       level: 13)
Coordinates:
  * latitude                                          (latitude) float32 3kB ...
  * level                                             (level) int64 104B 50 ....
  * longitude                                         (longitude) float32 6kB ...
  * time                                              (time) datetime64[ns] 748kB ...
Data variables: (12/62)
    10m_u_component_of_wind                           (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind                           (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_wind_speed                                    (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_dewpoint_temperature                           (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_temperature                                    (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    above_ground                                      (time, level, latitude, longitude) float32 5TB dask.array<chunksize=(1, 13, 721, 1440), meta=np.ndarray>
    ...                                                ...
    volumetric_soil_water_layer_1                     (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_2                     (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_3                     (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_4                     (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    vorticity                                         (time, level, latitude, longitude) float32 5TB dask.array<chunksize=(1, 13, 721, 1440), meta=np.ndarray>
    wind_speed                                        (time, level, latitude, longitude) float32 5TB dask.array<chunksize=(1, 13, 721, 1440), meta=np.ndarray>

In [6]:
# List of surface variable names
surface_vars = ['2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure']

# Select surface variables
surf_vars_ds = ds[surface_vars]


## Atmospheric variables

In [7]:
atmostpheric_variables = ["temperature", "u_component_of_wind", "v_component_of_wind", "specific_humidity", "geopotential"]
atmos_vars_ds = ds[atmostpheric_variables]

In [10]:
len(atmos_vars_ds.latitude)

721

# Static variables

In [10]:
static_variables = ["land_sea_mask", "soil_type", "geopotential_at_surface"]
static_vars_ds = ds[static_variables]

In [14]:
# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs

# ax = plt.axes(projection = ccrs.PlateCarree())

# mesh = ax.pcolormesh(df_era5.longitude.values
#                      ,df_era5.latitude.values,np.squeeze(df_era5.isel({'time':50,'level':0}).values))

# ax.coastlines()


In [15]:
# import xarray as xr
# import gcsfs
# import numpy as np
# import datetime

# fs = gcsfs.GCSFileSystem(token="anon")

# # Open the dataset from Google Cloud Storage
# store = fs.get_mapper('gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr')
# ds = xr.open_zarr(store=store, consolidated=True)

# # Define the time range
# start_date = np.datetime64('1959-01-01')
# end_date = np.datetime64('2023-01-11')

# # Define the geographical bounds
# lat_min = -22.125  # Minimum latitude
# lat_max = -37.875  # Maximum latitude
# lon_min = 15.125   # Minimum longitude
# lon_max = 34.875   # Maximum longitude

# # Select specific humidity for the defined time range and geographical area
# df_sa = ds.sel(
#     latitude=slice(lat_max, lat_min),  # Note: slicing from max to min for latitude
#     longitude=slice(lon_min, lon_max)
# )


In [16]:
# df_sa

In [17]:
# ds.variables

In [18]:
# # Example: Selecting specific variables
# surface_vars = ds[['2m_temperature', '10m_v_component_of_wind', '10m_wind_speed', 'mean_surface_latent_heat_flux']]
# static_vars = ds[['lsm', 'z', 'slt']]
# atmospheric_vars = ds[['z', 'u', 'v', 't', 'q']]


In [22]:
import torch
from torch.utils.data import Dataset
from aurora import Batch, Metadata

class ERA5ZarrDataset(Dataset):
    def __init__(self, surf_vars_ds, atmos_vars_ds, static_vars_ds, sequence_length):
        self.surf_vars_ds = surf_vars_ds
        self.atmos_vars_ds = atmos_vars_ds
        self.static_vars_ds = static_vars_ds
        self.sequence_length = sequence_length
        self.time_indices = range(sequence_length, len(surf_vars_ds.time))

    def __len__(self):
        return len(self.time_indices)

    def __getitem__(self, idx):
        i = self.time_indices[idx]
        time_slice = slice(i - self.sequence_length, i)

        surf_vars = {
            "2t": torch.from_numpy(self.surf_vars_ds["2m_temperature"].isel(time=time_slice).values[None]),
            "10u": torch.from_numpy(self.surf_vars_ds["10m_u_component_of_wind"].isel(time=time_slice).values[None]),
            "10v": torch.from_numpy(self.surf_vars_ds["10m_v_component_of_wind"].isel(time=time_slice).values[None]),
            "msl": torch.from_numpy(self.surf_vars_ds["mean_sea_level_pressure"].isel(time=time_slice).values[None]),
        }

        static_vars = {
            "z": torch.from_numpy(self.static_vars_ds["geopotential_at_surface"].values[0]),
            "slt": torch.from_numpy(self.static_vars_ds["soil_type"].values[0]),
            "lsm": torch.from_numpy(self.static_vars_ds["land_sea_mask"].values[0]),
        }

        atmos_vars = {
            "t": torch.from_numpy(self.atmos_vars_ds["temperature"].isel(time=time_slice).values[None]),
            "u": torch.from_numpy(self.atmos_vars_ds["u_component_of_wind"].isel(time=time_slice).values[None]),
            "v": torch.from_numpy(self.atmos_vars_ds["v_component_of_wind"].isel(time=time_slice).values[None]),
            "q": torch.from_numpy(self.atmos_vars_ds["specific_humidity"].isel(time=time_slice).values[None]),
            "z": torch.from_numpy(self.atmos_vars_ds["geopotential"].isel(time=time_slice).values[None]),
        }

        metadata = Metadata(
            lat=torch.from_numpy(self.surf_vars_ds.latitude.values),
            lon=torch.from_numpy(self.surf_vars_ds.longitude.values),
            time=tuple(self.surf_vars_ds.time.isel(time=time_slice).values.astype("datetime64[s]").tolist()),
            atmos_levels=tuple(int(level) for level in self.atmos_vars_ds.level.values),
        )

        return Batch(surf_vars=surf_vars, static_vars=static_vars, atmos_vars=atmos_vars, metadata=metadata)


In [23]:
my_batches = ERA5ZarrDataset(surf_vars_ds, atmos_vars_ds, static_vars_ds,2)

In [24]:
my_batches

In [26]:
# i=0
# for batch in my_batches:
#     i+=1
# i
len(my_batches)

93542

In [39]:
data = iter(my_batches)


In [46]:
data0 = next(data)

In [36]:
data0.surf_vars["2t"].shape

torch.Size([1, 2, 721, 1440])

In [55]:
data0.metadata.time

(datetime.datetime(1959, 1, 1, 18, 0), datetime.datetime(1959, 1, 2, 0, 0))